#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Ejercicio 4 CNN  Alumno: Ibarra Ramírez Sergio </font>

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

X,y = fetch_openml('mnist_784', as_frame=False, return_X_y=True)


# (Batches, ancho, alto, num_canales)
X = normalize(X).reshape(-1,28,28,1)

X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

codificador = OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train = codificador.fit_transform(y_train.reshape(-1,1))
y_test = codificador.transform(y_test.reshape(-1,1))

X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
batch_size = 100 # Hiperparámetro
train_dataset = train_dataset.shuffle(batch_size).batch(batch_size)

c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
# Initialize weights
b = tf.Variable(tf.zeros([10]))
W = tf.Variable(tf.random.truncated_normal([1568, 10], stddev=0.1)) 

conv1_filter = tf.Variable(tf.random.truncated_normal([4, 4, 1, 16], stddev=0.1)) # (alto,ancho,canales,num_filtros)
conv1_bias =  tf.Variable(tf.random.truncated_normal([16], stddev=0.1))

conv2_filter = tf.Variable(tf.random.truncated_normal([4, 4, 16, 32], stddev=0.1))
conv2_bias = tf.Variable(tf.random.truncated_normal([32], stddev=0.1))

# Training loop
optimizador = tf.keras.optimizers.experimental.SGD(learning_rate=0.005)


epochs = 10

for epoch in range(epochs):
  loss_epoch = 0
  for x, y in train_dataset:
    with tf.GradientTape() as gt:
    
      conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,2,2,1], padding='SAME')
      conv1_mas_bias = tf.nn.bias_add(conv1, conv1_bias)
      conv1_salida_activacion = tf.nn.relu(conv1_mas_bias)
      
      conv2 = tf.nn.conv2d(conv1_salida_activacion, conv2_filter, strides=[1,2,2,1], padding='SAME')  
      conv2_mas_bias = tf.nn.bias_add(conv2, conv2_bias)
      conv2_salida_activacion = tf.nn.relu(conv2_mas_bias)

      flat = tf.reshape(conv2_salida_activacion, [-1, 1568])
      
      dense = tf.matmul(flat, W) + b  
      
      
      x_ent = tf.nn.softmax_cross_entropy_with_logits(
          logits = dense,
          labels = y
      )
      
      loss = tf.reduce_mean(x_ent)
      optimizador.minimize(loss,[W,b,conv1_filter,b_conv1, conv2_filter, b_conv2],gt) # Poderosa

    loss_epoch += loss/len(train_dataset)
  print(loss_epoch)

tf.Tensor(2.3039744, shape=(), dtype=float32)
tf.Tensor(2.2832217, shape=(), dtype=float32)
tf.Tensor(2.257505, shape=(), dtype=float32)
tf.Tensor(2.1964169, shape=(), dtype=float32)
tf.Tensor(2.0284784, shape=(), dtype=float32)
tf.Tensor(1.5714421, shape=(), dtype=float32)
tf.Tensor(0.9279587, shape=(), dtype=float32)
tf.Tensor(0.6191782, shape=(), dtype=float32)
tf.Tensor(0.5079457, shape=(), dtype=float32)
tf.Tensor(0.45420605, shape=(), dtype=float32)


Se aplica la red CNN para la clasificacion de imagenes de numeros 

In [14]:
conv1 = tf.nn.conv2d(X_test, conv1_filter, strides=[1,2,2,1], padding='SAME')
conv1_mas_bias = tf.nn.bias_add(conv1, conv1_bias)
conv1_salida_activacion = tf.nn.relu(conv1_mas_bias)
      
conv2 = tf.nn.conv2d(conv1_salida_activacion, conv2_filter, strides=[1,2,2,1], padding='SAME')  
conv2_mas_bias = tf.nn.bias_add(conv2, conv2_bias)
conv2_salida_activacion = tf.nn.relu(conv2_mas_bias)

flat = tf.reshape(conv2_salida_activacion, [-1, 1568])
      
dense = tf.matmul(flat, W) + b  


probs = tf.nn.softmax(dense)

max_preds = tf.argmax(probs,1)
max_trues = tf.argmax(y_test,1)

numCorrectos = tf.equal(max_preds,max_trues)
acc = tf.reduce_mean(tf.cast(numCorrectos,tf.float32)) # Tengo que convertirlo porque el original es booleano
print(acc)
      
      
      

tf.Tensor(0.883, shape=(), dtype=float32)


### Implementacion de CNN en tf.keras 

In [2]:
entradas = tf.keras.Input(shape=(28,28,1))
capaConv = tf.keras.layers.Conv2D(16,4,2,
                                  "same",
                                  activation = "relu")(entradas)
aplanado = tf.keras.layers.Reshape((-1,))(capaConv)
capaSalida = tf.keras.layers.Dense(10,activation="softmax")(aplanado)

In [3]:
modelo_CNN1 = tf.keras.Model(inputs=entradas,outputs=capaSalida)

modelo_CNN1.compile(optimizer="SGD",
               loss="categorical_crossentropy",
               metrics=["accuracy"])

In [4]:
modelo_CNN1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 16)        272       
                                                                 
 reshape (Reshape)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 10)                31370     
                                                                 
Total params: 31642 (123.60 KB)
Trainable params: 31642 (123.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
modelo_CNN1.fit(X_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
600/600 [==============================] - 12s 9ms/step - loss: 2.2891 - accuracy: 0.1233
Epoch 2/10
600/600 [==============================] - 5s 8ms/step - loss: 2.2054 - accuracy: 0.3525
Epoch 3/10
600/600 [==============================] - 5s 8ms/step - loss: 1.8176 - accuracy: 0.6779
Epoch 4/10
600/600 [==============================] - 5s 9ms/step - loss: 1.1143 - accuracy: 0.7983
Epoch 5/10
600/600 [==============================] - 6s 10ms/step - loss: 0.7177 - accuracy: 0.8417
Epoch 6/10
600/600 [==============================] - 6s 10ms/step - loss: 0.5624 - accuracy: 0.8617
Epoch 7/10
600/600 [==============================] - 6s 10ms/step - loss: 0.4882 - accuracy: 0.8731
Epoch 8/10
600/600 [==============================] - 7s 11ms/step - loss: 0.4458 - accuracy: 0.8792
Epoch 9/10
600/600 [==============================] - 6s 11ms/step - loss: 0.4185 - accuracy: 0.8837
Epoch 10/10
600/600 [==============================] - 6s 10ms/step - loss: 0.3996 - accuracy:

Vamos a evaluar el modelo CNN construido con tf.keras para la clasificacion de digitos en forma de imagenes del MNIST 

In [6]:
loss_modelo_CNN1, acc_modelo_CNN1 = modelo_CNN1.evaluate(X_test,y_test)

print(acc_modelo_CNN1)

313/313 [==============================] - 2s 3ms/step - loss: 0.3699 - accuracy: 0.8990
0.8989999890327454


### Modelo secuencial de Implementacion de CNN en tf.keras 

In [7]:
modelo_CNN_sequencial = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,4,2,"same",activation="relu",input_shape=(28,28,1)),
    tf.keras.layers.Reshape((-1,)),
    tf.keras.layers.Dense(10,activation="softmax")
])

modelo_CNN_sequencial.compile(optimizer="SGD",
               loss="categorical_crossentropy",
               metrics=["accuracy"])

modelo_CNN_sequencial.fit(X_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
600/600 [==============================] - 8s 11ms/step - loss: 2.2871 - accuracy: 0.1451
Epoch 2/10
600/600 [==============================] - 7s 11ms/step - loss: 2.2184 - accuracy: 0.3244
Epoch 3/10
600/600 [==============================] - 6s 10ms/step - loss: 1.9123 - accuracy: 0.6443
Epoch 4/10
600/600 [==============================] - 6s 10ms/step - loss: 1.2394 - accuracy: 0.7868
Epoch 5/10
600/600 [==============================] - 6s 10ms/step - loss: 0.7742 - accuracy: 0.8354
Epoch 6/10
600/600 [==============================] - 6s 10ms/step - loss: 0.5884 - accuracy: 0.8563
Epoch 7/10
600/600 [==============================] - 6s 10ms/step - loss: 0.5027 - accuracy: 0.8699
Epoch 8/10
600/600 [==============================] - 6s 10ms/step - loss: 0.4551 - accuracy: 0.8766
Epoch 9/10
600/600 [==============================] - 6s 10ms/step - loss: 0.4250 - accuracy: 0.8822
Epoch 10/10
600/600 [==============================] - 6s 10ms/step - loss: 0.4043 - accura

Vamos a evaluar el modelo CNN sequencial construido con tf.keras para la clasificacion de digitos en forma de imagenes del MNIST 

In [8]:
loss_modelo_CNN_sequencial, acc_modelo_CNN_sequencial = modelo_CNN_sequencial.evaluate(X_test,y_test)

print(acc_modelo_CNN_sequencial)

313/313 [==============================] - 1s 3ms/step - loss: 0.3754 - accuracy: 0.8946
0.894599974155426


### Poolin en la Implementacion de CNN con tf.keras 

In [9]:
modelo_CNN_pooling = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,4,1,"same",activation="relu",input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2,strides=2,padding="valid"),
    tf.keras.layers.Reshape((-1,)),
    tf.keras.layers.Dense(10,activation="softmax")
])

modelo_CNN_pooling.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 16)        272       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 reshape_2 (Reshape)         (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                31370     
                                                                 
Total params: 31642 (123.60 KB)
Trainable params: 31642 (123.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
modelo_CNN_pooling.compile(optimizer="SGD",
               loss="categorical_crossentropy",
               metrics=["accuracy"])

modelo_CNN_pooling.fit(X_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
600/600 [==============================] - 27s 30ms/step - loss: 2.2853 - accuracy: 0.1651
Epoch 2/10
600/600 [==============================] - 18s 31ms/step - loss: 2.1589 - accuracy: 0.5268
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 1.5967 - accuracy: 0.7413
Epoch 4/10
600/600 [==============================] - 22s 36ms/step - loss: 0.9133 - accuracy: 0.8170
Epoch 5/10
600/600 [==============================] - 22s 37ms/step - loss: 0.6355 - accuracy: 0.8486
Epoch 6/10
600/600 [==============================] - 23s 38ms/step - loss: 0.5241 - accuracy: 0.8656
Epoch 7/10
600/600 [==============================] - 21s 35ms/step - loss: 0.4670 - accuracy: 0.8759
Epoch 8/10
600/600 [==============================] - 17s 29ms/step - loss: 0.4323 - accuracy: 0.8813
Epoch 9/10
600/600 [==============================] - 23s 38ms/step - loss: 0.4091 - accuracy: 0.8860
Epoch 10/10
600/600 [==============================] - 25s 42ms/step - loss: 0.392

Vamos a evaluar el modelo CNN con Pooling construido con tf.keras para la clasificacion de digitos en forma de imagenes del MNIST 

In [11]:
loss_modelo_CNN_pooling, acc_modelo_CNN_pooling = modelo_CNN_pooling.evaluate(X_test,y_test)

print(acc_modelo_CNN_pooling)

313/313 [==============================] - 3s 6ms/step - loss: 0.3652 - accuracy: 0.8985
0.8985000252723694
